In [55]:
import numpy as np
import pandas as pd
import datetime

In [66]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.filter(['subject_id','hadm_id','icustay_id','icu_day','chloride_max','chloride_input_meq'])
df= df.query('icu_day == 1')
# df = df.dropna(subset = ['chloride_max'])

df_chl_max = df.filter(['subject_id','hadm_id','icustay_id','chloride_max'])
df_chl_max = df_chl_max.dropna(subset = ['chloride_max'])
df_chl_input = df.filter(['subject_id','hadm_id','icustay_id','chloride_input_meq'])

df_chl_max = df_chl_max.set_index(['subject_id','hadm_id','icustay_id'])
df_chl_input = df_chl_input.set_index(['subject_id','hadm_id','icustay_id'])

In [71]:
# Target Group: adult patients' first icu visit
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)

# Create ICU Id list
icu = []
for icuid in df['icustay_id']:
    icu.append(icuid)

    
# Max chloride
df = pd.read_csv('chloride.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.filter(['subject_id','hadm_id','icustay_id','icu_day','chloride_max','chloride_input_meq'])
df= df.query('icu_day == 1')
# df = df.dropna(subset = ['chloride_max'])

df_chl_max = df.filter(['subject_id','hadm_id','icustay_id','chloride_max'])
df_chl_max = df_chl_max.dropna(subset = ['chloride_max'])
df_chl_input = df.filter(['subject_id','hadm_id','icustay_id','chloride_input_meq'])


df_chl_max = df_chl_max.set_index(['subject_id','hadm_id','icustay_id'])
df_chl_input = df_chl_input.set_index(['subject_id','hadm_id','icustay_id'])

# Max chloride
#df = pd.read_csv('chloride.csv')
#df = df[df['icustay_id'].isin(icu)]
#df = df.set_index(['subject_id','hadm_id','icustay_id'])
#df = df.filter(['icustay_id','icu_day','chloride_max','chloride_input_meq'])
# df = df.dropna(subset = ['chloride_max'])

# Chloride >= 110 in the second day
#df_chl = df.query('icu_day == 2')
#df_chl['chl_110'] = (df_chl['chloride_max'] >= 110).astype(int)

# Demographic
df = pd.read_csv('adm_demographics.csv')
df = df.set_index(['subject_id', 'hadm_id'])
df_demo = df.filter(['insurance','ethnicity','age','gender'])

# Weight
df = pd.read_csv('weight.csv')
df = df[df['icustay_id'].isin(icu)]
df_wt = df.filter(['icustay_id','day','weight'])

# GCS(Glasgow coma scale)
df = pd.read_csv('gcs_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df_gcs = df.filter(['icustay_id','day','mingcs'])

# First-day Vitals(Merge with GCS and Weight)
df = pd.read_csv('vitals_pan.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.set_index(['subject_id','hadm_id','icustay_id'])
df_vitals = df.filter(['day', 'heartrate_max','sysbp_min','diasbp_min','resprate_max','spo2_min','tempc_max'])
df_vitals = df_vitals.reset_index()
df_vitals = df_vitals.merge(df_wt, on = ['icustay_id','day']).merge(df_gcs, on = ['icustay_id','day'])
df_vitals = df_vitals.query('(day == 1) & (0 < heartrate_max <= 250) &(40<sysbp_min <190) &(20<diasbp_min<150) &(6<resprate_max<80) &(0<spo2_min<100)&(30<tempc_max<44)&(30<weight<500)&(3<mingcs<15)'
                           )
df_vitals = df_vitals.set_index(['subject_id','hadm_id','icustay_id'])

# Medication from prescription table
df = pd.read_csv('medication.csv')
df_med = df.drop(df.columns[[0,3]], axis=1)
df = pd.read_csv('icu_first_18.csv')
df = df.drop(df.columns[0], axis=1)
df = df_med.merge(df, on = ['subject_id','hadm_id'])
df['taken'] = ((pd.to_datetime(df['intime']) <= pd.to_datetime(df['startdate'])) & 
               ((pd.to_datetime(df['startdate']) <= (pd.to_datetime(df['intime']) + datetime.timedelta(days=1)))))
df = df.filter(['subject_id','hadm_id','icustay_id','drug','taken'])
df_med = df.set_index(['subject_id','hadm_id','icustay_id'])
df_med = df_med.groupby(['subject_id','hadm_id','icustay_id','drug']).any()
df_med = df_med['taken'].unstack()
df_med = (df_med * 1).fillna(0)

# Intake and output
df = pd.read_csv('fluid_io.csv')
df = df[df['icustay_id'].isin(icu)]
df = df.query('icu_day == 1')
df_fluid = df.set_index(['subject_id','hadm_id','icustay_id'])
df_fluid.head()

# Diagnosis Code(group by elixhauser)
df = pd.read_csv('elixhauser.csv')
df_icd = df.drop(df.columns[[0]], axis=1)
df_icd = df_icd.set_index('hadm_id')

# Labs
df = pd.read_csv('labs1.csv')
df = df.drop(df.columns[0], axis=1)
df = df[df['icustay_id'].isin(icu)]
df_labs = df.query('day == 1 & 0.1<lactate_max<40 & 90<sodium_max<190 & 1.5<potassium_max<15 & 1<albumin_max<6.5 & 5<bicarbonate_min<65 & 0.1<creatinine_max<47 & 66<chloride_max<170 & 18<glucose_max<2500 & 2<hemoglobin_min<21 & 5<platelet_min<1600  & 0.5<inr_max<20 & 1<bun_max<280 & 0.1<wbc_max<680 & 0.1<bilirubin_min<80 & 1.5<calcium_total_min<18 & 0.2<calcium_ionized_min<2.5 & 2<ast_sgot_max<25000 & 3<amylase_max<25000 &  1<lipase_max<25000 & 0.1<c_reactive_protein_max<300')
df_labs = df_labs.set_index(['subject_id','hadm_id','icustay_id'])

# Interventions
df = pd.read_csv('interventions.csv')
df = df.query('day == 1')
df_int = df.drop(df.columns[[0]], axis=1)
df_int = df_int.set_index(['subject_id', 'hadm_id', 'icustay_id'])
df_int = df_int.filter(['label'])
df_int['taken'] = 1
df_int = df_int.groupby(['subject_id','hadm_id','icustay_id','label']).any()
df_int = df_int['taken'].unstack()
df_int = (df_int * 1).fillna(0)

In [75]:
demo = ['age', 'gender', 'ethnicity']
vitals = ['heartrate_max', 'sysbp_min', 'diasbp_min', 'resprate_max', 'weight', 'mingcs']
medication = ['norepinephrine']
fluid = ['fluid_net_input_ml']
diag = ['congestive_heart_failure',
       'cardiac_arrhythmias', 'valvular_disease', 'pulmonary_circulation',
       'peripheral_vascular', 'hypertension', 
        'chronic_pulmonary', 
       'diabetes_complicated',  'renal_failure',
       'liver_disease',  
        'coagulopathy',  'fluid_electrolyte',
       'blood_loss_anemia']
labs = [ #'chloride_max',
    'sodium_max', 'bicarbonate_min']
interventions = ['EPAP','IPAP','LPM','MeanAirwayPressure']

In [76]:
df_demo = df_demo[demo]
df_vitals = df_vitals[vitals]
df_med = df_med[medication]
df_fluid = df_fluid[fluid]
df_icd = df_icd[diag]
df_labs = df_labs[labs]
df_int = df_int[interventions]
#df_chl = df_chl[['chl_110','chloride_input_meq']]

In [87]:
df = df_chl_max.join(df_demo, how = 'inner') \
.join(df_chl_input, how = 'left') \
.join(df_fluid, how = 'left') \
.join(df_icd, how = 'left') \
.join(df_labs, how = 'left') \
.join(df_int, how='left') \
.join(df_vitals, how='left') \
.join(df_med, how = 'left') 

df = df.fillna(0)
df

chloride_max        age gender ethnicity  \
subject_id hadm_id icustay_id                                             
3          145834  211552             114.0  76.575342      M     WHITE   
4          185777  294638             108.0  47.876712      F     WHITE   
6          107064  228232             101.0  65.983562      F     WHITE   
9          150750  220597             103.0  41.816438      M     OTHER   
11         194540  229441             106.0  50.180822      F     WHITE   
12         112213  232669             113.0  72.419178      M     WHITE   
13         143045  263738             116.0  39.890411      F     WHITE   
17         161087  257980             113.0  47.849315      F     WHITE   
           194023  277042             112.0  47.484932      F     WHITE   
18         188822  298129             102.0  50.873973      M     WHITE   
19         109235  273430             105.0  91.400000      M     WHITE   
20         157681  264490             108.0  75.926027      F     WHITE   
21         109451  217847             104.0  87.495890      M     WHITE   
           111970  216859             104.0  87.882192      M     WHITE   
22         165315  204798             105.0  64.969863      F     WHITE   
23         124321  234044             108.0  75.304110      M     WHITE   
           152223  227807             114.0  71.178082      M     WHITE   
24         161859  262236             107.0  39.041096      M     WHITE   
25         129635  203487             111.0  58.989041      M     WHITE   
26         197661  244882             106.0  72.052055      M     OTHER   
28         162569  225559             112.0  74.432877      M     WHITE   
30         104557  225176              97.0  91.400000      M     OTHER   
31         128652  254478             103.0  72.312329      M     WHITE   
32         175413  295037             104.0  45.542466      M     WHITE   
33         176176  296681             106.0  82.446575      M     OTHER   
34         115799  263086             105.0  91.400000      M     WHITE   
           144319  290505             107.0  91.400000      M     WHITE   
35         166707  282039             120.0  69.487671      F     WHITE   
36         122659  211200             109.0  69.778082      M     WHITE   
           165660  241249             109.0  72.775342      M     WHITE   
...                                     ...        ...    ...       ...   
99897      181057  298720             101.0  54.493151      M     BLACK   
99899      188409  245756              92.0  87.013699      M     BLACK   
99901      131711  226541              86.0  68.873973      M     WHITE   
99903      135939  291066             109.0  50.142466      M     BLACK   
99912      189380  236183             112.0  84.600000      M     WHITE   
99913      175989  292375             112.0  71.136986      M  HISPANIC   
99922      123563  259919             112.0  69.326027      M     WHITE   
99923      164914  202505              92.0  54.424658      M     WHITE   
           192053  298857             108.0  54.646575      M     WHITE   
99928      104802  236704             104.0  20.884932      F  HISPANIC   
99936      107913  213906             107.0  91.400000      F     WHITE   
99937      129380  291004             103.0  65.720548      F  HISPANIC   
99938      103815  296707             107.0  80.890411      M     WHITE   
99939      159023  252442             107.0  26.967123      M  HISPANIC   
99944      185654  221067             106.0  85.536986      F     WHITE   
99946      157197  298613             105.0  49.698630      M     WHITE   
99955      108494  243255             115.0  63.232877      M     BLACK   
99957      148574  257127              98.0  68.095890      M     WHITE   
99965      101083  257338             108.0  66.334247      M     WHITE   
99966      167228  252173             106.0  77.536986      M     WHITE   
99973      150202  275083             117.0  65.542466  

In [88]:
df['gender'] = (df['gender'] == 'F').astype(int)

In [89]:
df = pd.concat([df, pd.get_dummies(df['ethnicity'])], axis=1)
df = df.drop(['ethnicity', 'OTHER'], axis=1)
df

chloride_max        age  gender  \
subject_id hadm_id icustay_id                                    
3          145834  211552             114.0  76.575342       0   
4          185777  294638             108.0  47.876712       1   
6          107064  228232             101.0  65.983562       1   
9          150750  220597             103.0  41.816438       0   
11         194540  229441             106.0  50.180822       1   
12         112213  232669             113.0  72.419178       0   
13         143045  263738             116.0  39.890411       1   
17         161087  257980             113.0  47.849315       1   
           194023  277042             112.0  47.484932       1   
18         188822  298129             102.0  50.873973       0   
19         109235  273430             105.0  91.400000       0   
20         157681  264490             108.0  75.926027       1   
21         109451  217847             104.0  87.495890       0   
           111970  216859             104.0  87.882192       0   
22         165315  204798             105.0  64.969863       1   
23         124321  234044             108.0  75.304110       0   
           152223  227807             114.0  71.178082       0   
24         161859  262236             107.0  39.041096       0   
25         129635  203487             111.0  58.989041       0   
26         197661  244882             106.0  72.052055       0   
28         162569  225559             112.0  74.432877       0   
30         104557  225176              97.0  91.400000       0   
31         128652  254478             103.0  72.312329       0   
32         175413  295037             104.0  45.542466       0   
33         176176  296681             106.0  82.446575       0   
34         115799  263086             105.0  91.400000       0   
           144319  290505             107.0  91.400000       0   
35         166707  282039             120.0  69.487671       1   
36         122659  211200             109.0  69.778082       0   
           165660  241249             109.0  72.775342       0   
...                                     ...        ...     ...   
99897      181057  298720             101.0  54.493151       0   
99899      188409  245756              92.0  87.013699       0   
99901      131711  226541              86.0  68.873973       0   
99903      135939  291066             109.0  50.142466       0   
99912      189380  236183             112.0  84.600000       0   
99913      175989  292375             112.0  71.136986       0   
99922      123563  259919             112.0  69.326027       0   
99923      164914  202505              92.0  54.424658       0   
           192053  298857             108.0  54.646575       0   
99928      104802  236704             104.0  20.884932       1   
99936      107913  213906             107.0  91.400000       1   
99937      129380  291004             103.0  65.720548       1   
99938      103815  296707             107.0  80.890411       0   
99939      159023  252442             107.0  26.967123       0   
99944      185654  221067             106.0  85.536986       1   
99946      157197  298613             105.0  49.698630       0   
99955      108494  243255             115.0  63.232877       0   
99957      148574  257127              98.0  68.095890       0   
99965      101083  257338             108.0  66.334247       0   
99966      167228  252173             106.0  77.536986       0   
99973      150202  275083             117.0  65.542466       1   
99982      112748  275958              99.0  65.304110       0   
           151454  221194             100.0  65.200000       0   
           183791  239827              92.0  65.419178       0   
99983      117390  286606             106.0  78.627397       0   
99985      176670  279638             105.0  53.843836       0   
99991      151118  226241             114.0  47.747945       0   
99992      197084  242052             104.0  65.813699       1   
99995

## standarize all values to z-score

In [83]:
stan = pd.read_csv('standard.csv')
stan = stan.set_index('vitalid')
stan

,mean,median,std,iqr
vitalid,,,,
BICARBONATE,25.696443,25.000000,5.034271,7.000000
CHLORIDE,104.144561,104.000000,6.322994,8.000000
SODIUM,138.852473,139.000000,5.119334,6.000000
DiasBP,59.431493,58.000000,14.794056,19.000000
HeartRate,82.917012,82.000000,16.126807,22.000000
RespRate,19.693027,19.000000,5.462299,7.000000
SysBP,123.279089,121.000000,22.821488,31.000000
mingcs,13.662176,15.000000,2.335564,1.000000
age,64.951495,66.671233,17.048611,24.665753


In [84]:
df=df.replace({'heartrate_max': {0: stan.loc['HeartRate']['mean']},
               'sysbp_min': {0: stan.loc['SysBP']['mean']},
               'diasbp_min': {0: stan.loc['DiasBP']['mean']},
               'resprate_max': {0: stan.loc['RespRate']['mean']},
               'heartrate_max': {0: stan.loc['HeartRate']['mean']},
               'weight': {0: df_wt['weight'].mean()},
               'mingcs': {0: stan.loc['mingcs']['mean']},
               'chloride_max': {0: stan.loc['CHLORIDE']['mean']},
               'sodium_max': {0: stan.loc['SODIUM']['mean']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['mean']}
              
              }) 
df

chloride_max        age  gender  \
subject_id hadm_id icustay_id                                    
3          145834  211552             114.0  76.575342       0   
4          185777  294638             108.0  47.876712       1   
6          107064  228232             101.0  65.983562       1   
9          150750  220597             103.0  41.816438       0   
11         194540  229441             106.0  50.180822       1   
12         112213  232669             113.0  72.419178       0   
13         143045  263738             116.0  39.890411       1   
17         161087  257980             113.0  47.849315       1   
           194023  277042             112.0  47.484932       1   
18         188822  298129             102.0  50.873973       0   
19         109235  273430             105.0  91.400000       0   
20         157681  264490             108.0  75.926027       1   
21         109451  217847             104.0  87.495890       0   
           111970  216859             104.0  87.882192       0   
22         165315  204798             105.0  64.969863       1   
23         124321  234044             108.0  75.304110       0   
           152223  227807             114.0  71.178082       0   
24         161859  262236             107.0  39.041096       0   
25         129635  203487             111.0  58.989041       0   
26         197661  244882             106.0  72.052055       0   
28         162569  225559             112.0  74.432877       0   
30         104557  225176              97.0  91.400000       0   
31         128652  254478             103.0  72.312329       0   
32         175413  295037             104.0  45.542466       0   
33         176176  296681             106.0  82.446575       0   
34         115799  263086             105.0  91.400000       0   
           144319  290505             107.0  91.400000       0   
35         166707  282039             120.0  69.487671       1   
36         122659  211200             109.0  69.778082       0   
           165660  241249             109.0  72.775342       0   
...                                     ...        ...     ...   
99897      181057  298720             101.0  54.493151       0   
99899      188409  245756              92.0  87.013699       0   
99901      131711  226541              86.0  68.873973       0   
99903      135939  291066             109.0  50.142466       0   
99912      189380  236183             112.0  84.600000       0   
99913      175989  292375             112.0  71.136986       0   
99922      123563  259919             112.0  69.326027       0   
99923      164914  202505              92.0  54.424658       0   
           192053  298857             108.0  54.646575       0   
99928      104802  236704             104.0  20.884932       1   
99936      107913  213906             107.0  91.400000       1   
99937      129380  291004             103.0  65.720548       1   
99938      103815  296707             107.0  80.890411       0   
99939      159023  252442             107.0  26.967123       0   
99944      185654  221067             106.0  85.536986       1   
99946      157197  298613             105.0  49.698630       0   
99955      108494  243255             115.0  63.232877       0   
99957      148574  257127              98.0  68.095890       0   
99965      101083  257338             108.0  66.334247       0   
99966      167228  252173             106.0  77.536986       0   
99973      150202  275083             117.0  65.542466       1   
99982      112748  275958              99.0  65.304110       0   
           151454  221194             100.0  65.200000       0   
           183791  239827              92.0  65.419178       0   
99983      117390  286606             106.0  78.627397       0   
99985      176670  279638             105.0  53.843836       0   
99991      151118  226241             114.0  47.747945       0   
99992      197084  242052             104.0  65.813699       1   
99995

In [85]:
df['heartrate_max'] = (df['heartrate_max'] - stan.loc['HeartRate']['mean'])/stan.loc['HeartRate']['std']
df['sysbp_min'] = (df['sysbp_min'] - stan.loc['SysBP']['mean'])/stan.loc['SysBP']['std']
df['diasbp_min'] = (df['diasbp_min'] - stan.loc['DiasBP']['mean'])/stan.loc['DiasBP']['std']
df['resprate_max'] = (df['resprate_max'] - stan.loc['RespRate']['mean'])/stan.loc['RespRate']['std']
df['weight'] = (df['weight'] - df_wt['weight'].mean())/df_wt['weight'].std()
df['mingcs'] = (df['mingcs'] - stan.loc['mingcs']['mean'])/stan.loc['mingcs']['std']

df['chloride_max'] = (df['chloride_max'] - stan.loc['CHLORIDE']['mean'])/stan.loc['CHLORIDE']['std']
df['sodium_max'] = (df['sodium_max'] - stan.loc['SODIUM']['mean'])/stan.loc['SODIUM']['std']
df['bicarbonate_min'] = (df['bicarbonate_min'] - stan.loc['BICARBONATE']['mean'])/stan.loc['BICARBONATE']['std']


In [86]:
df['fluid_net_input_ml'] = (df['fluid_net_input_ml'] - df['fluid_net_input_ml'].mean())/ df['fluid_net_input_ml'].std()
df

chloride_max        age  gender  \
subject_id hadm_id icustay_id                                    
3          145834  211552          1.558667  76.575342       0   
4          185777  294638          0.609749  47.876712       1   
6          107064  228232         -0.497322  65.983562       1   
9          150750  220597         -0.181016  41.816438       0   
11         194540  229441          0.293443  50.180822       1   
12         112213  232669          1.400514  72.419178       0   
13         143045  263738          1.874972  39.890411       1   
17         161087  257980          1.400514  47.849315       1   
           194023  277042          1.242361  47.484932       1   
18         188822  298129         -0.339169  50.873973       0   
19         109235  273430          0.135290  91.400000       0   
20         157681  264490          0.609749  75.926027       1   
21         109451  217847         -0.022863  87.495890       0   
           111970  216859         -0.022863  87.882192       0   
22         165315  204798          0.135290  64.969863       1   
23         124321  234044          0.609749  75.304110       0   
           152223  227807          1.558667  71.178082       0   
24         161859  262236          0.451596  39.041096       0   
25         129635  203487          1.084208  58.989041       0   
26         197661  244882          0.293443  72.052055       0   
28         162569  225559          1.242361  74.432877       0   
30         104557  225176         -1.129933  91.400000       0   
31         128652  254478         -0.181016  72.312329       0   
32         175413  295037         -0.022863  45.542466       0   
33         176176  296681          0.293443  82.446575       0   
34         115799  263086          0.135290  91.400000       0   
           144319  290505          0.451596  91.400000       0   
35         166707  282039          2.507584  69.487671       1   
36         122659  211200          0.767902  69.778082       0   
           165660  241249          0.767902  72.775342       0   
...                                     ...        ...     ...   
99897      181057  298720         -0.497322  54.493151       0   
99899      188409  245756         -1.920698  87.013699       0   
99901      131711  226541         -2.869616  68.873973       0   
99903      135939  291066          0.767902  50.142466       0   
99912      189380  236183          1.242361  84.600000       0   
99913      175989  292375          1.242361  71.136986       0   
99922      123563  259919          1.242361  69.326027       0   
99923      164914  202505         -1.920698  54.424658       0   
           192053  298857          0.609749  54.646575       0   
99928      104802  236704         -0.022863  20.884932       1   
99936      107913  213906          0.451596  91.400000       1   
99937      129380  291004         -0.181016  65.720548       1   
99938      103815  296707          0.451596  80.890411       0   
99939      159023  252442          0.451596  26.967123       0   
99944      185654  221067          0.293443  85.536986       1   
99946      157197  298613          0.135290  49.698630       0   
99955      108494  243255          1.716819  63.232877       0   
99957      148574  257127         -0.971780  68.095890       0   
99965      101083  257338          0.609749  66.334247       0   
99966      167228  252173          0.293443  77.536986       0   
99973      150202  275083          2.033125  65.542466       1   
99982      112748  275958         -0.813627  65.304110       0   
           151454  221194         -0.655475  65.200000       0   
           183791  239827         -1.920698  65.419178       0   
99983      117390  286606          0.293443  78.627397       0   
99985      176670  279638          0.135290  53.843836       0   
99991      151118  226241          1.558667  47.747945       0   
99992      197084  242052         -0.022863  65.813699       1   
99995

In [12]:
df.to_csv('data.csv')

# Normalization

## impute with median

In [90]:
df1 =df.replace({'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'sysbp_min': {0: stan.loc['SysBP']['median']},
               'diasbp_min': {0: stan.loc['DiasBP']['median']},
               'resprate_max': {0: stan.loc['RespRate']['median']},
               'heartrate_max': {0: stan.loc['HeartRate']['median']},
               'weight': {0: df_wt['weight'].median()},
               'mingcs': {0: stan.loc['mingcs']['median']},
               'chloride_max': {0: stan.loc['CHLORIDE']['median']},
               'sodium_max': {0: stan.loc['SODIUM']['median']},
               'bicarbonate_min': {0: stan.loc['BICARBONATE']['median']}
              
              }) 
df1.head()

,,,chloride_max,age,gender,chloride_input_meq,fluid_net_input_ml,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,...,sysbp_min,diasbp_min,resprate_max,weight,mingcs,norepinephrine,ASIAN,BLACK,HISPANIC,WHITE
subject_id,hadm_id,icustay_id,,,,,,,,,,,,,,,,,,,,,
3,145834,211552,114.0,76.575342,0,1861.475,0.000000,1.0,0.0,0.0,0.0,0.0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
4,185777,294638,108.0,47.876712,1,150.150,1015.000000,0.0,0.0,0.0,0.0,0.0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
6,107064,228232,101.0,65.983562,1,335.720,3842.500000,0.0,0.0,0.0,0.0,0.0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1
9,150750,220597,103.0,41.816438,0,51.590,1516.133019,1.0,0.0,0.0,0.0,0.0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,0
11,194540,229441,106.0,50.180822,1,35.550,900.000000,0.0,0.0,0.0,0.0,0.0,...,121.0,58.0,19.0,80.0,15.0,0.0,0,0,0,1


In [91]:
df1.to_csv('data_original_median.csv')

## normalize

In [92]:
df1['heartrate_max'] = (df1['heartrate_max'] - stan.loc['HeartRate']['median'])/stan.loc['HeartRate']['iqr']
df1['sysbp_min'] = (df1['sysbp_min'] - stan.loc['SysBP']['median'])/stan.loc['SysBP']['iqr']
df1['diasbp_min'] = (df1['diasbp_min'] - stan.loc['DiasBP']['median'])/stan.loc['DiasBP']['iqr']
df1['resprate_max'] = (df1['resprate_max'] - stan.loc['RespRate']['median'])/stan.loc['RespRate']['iqr']
df1['weight'] = (df1['weight'] - df_wt['weight'].median())/(df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25))
df1['mingcs'] = (df1['mingcs'] - stan.loc['mingcs']['median'])/stan.loc['mingcs']['iqr']

df1['chloride_max'] = (df1['chloride_max'] - stan.loc['CHLORIDE']['median'])/stan.loc['CHLORIDE']['iqr']
df1['sodium_max'] = (df1['sodium_max'] - stan.loc['SODIUM']['median'])/stan.loc['SODIUM']['iqr']
df1['bicarbonate_min'] = (df1['bicarbonate_min'] - stan.loc['BICARBONATE']['median'])/stan.loc['BICARBONATE']['iqr']

In [94]:
print(df1['age'].median(), (df1['age'].quantile(0.75) - df1['age'].quantile(0.25)),df1['age'].mean(),df1['age'].std() )

65.7479452054795 24.945205479452092 64.15323589540823 17.215711139799687


In [95]:
print(df_wt['weight'].median(), (df_wt['weight'].quantile(0.75) - df_wt['weight'].quantile(0.25)),df_wt['weight'].mean(),df_wt['weight'].std() )

80.0 28.0 83.06121690261101 24.71410967312458


In [96]:
df1['fluid_net_input_ml'] = (df1['fluid_net_input_ml'] - df1['fluid_net_input_ml'].median())/ (df1['fluid_net_input_ml'].quantile(0.75) - df1['fluid_net_input_ml'].quantile(0.25))
df1['age'] = (df1['age'] - df1['age'].median())/ (df1['age'].quantile(0.75) - df1['age'].quantile(0.25))

df1

chloride_max       age  gender  \
subject_id hadm_id icustay_id                                   
3          145834  211552             1.250  0.434047       0   
4          185777  294638             0.500 -0.716420       1   
6          107064  228232            -0.375  0.009445       1   
9          150750  220597            -0.125 -0.959363       0   
11         194540  229441             0.250 -0.624053       1   
12         112213  232669             1.125  0.267435       0   
13         143045  263738             1.500 -1.036573       1   
17         161087  257980             1.125 -0.717518       1   
           194023  277042             1.000 -0.732125       1   
18         188822  298129            -0.250 -0.596266       0   
19         109235  273430             0.125  1.028336       0   
20         157681  264490             0.500  0.408018       1   
21         109451  217847             0.000  0.871829       0   
           111970  216859             0.000  0.887315       0   
22         165315  204798             0.125 -0.031192       1   
23         124321  234044             0.500  0.383086       0   
           152223  227807             1.250  0.217683       0   
24         161859  262236             0.375 -1.070621       0   
25         129635  203487             0.875 -0.270950       0   
26         197661  244882             0.250  0.252718       0   
28         162569  225559             1.000  0.348160       0   
30         104557  225176            -0.875  1.028336       0   
31         128652  254478            -0.125  0.263152       0   
32         175413  295037             0.000 -0.809995       0   
33         176176  296681             0.250  0.669412       0   
34         115799  263086             0.125  1.028336       0   
           144319  290505             0.375  1.028336       0   
35         166707  282039             2.000  0.149918       1   
36         122659  211200             0.625  0.161560       0   
           165660  241249             0.625  0.281713       0   
...                                     ...       ...     ...   
99897      181057  298720            -0.375 -0.451181       0   
99899      188409  245756            -1.500  0.852499       0   
99901      131711  226541            -2.250  0.125316       0   
99903      135939  291066             0.625 -0.625590       0   
99912      189380  236183             1.000  0.755739       0   
99913      175989  292375             1.000  0.216035       0   
99922      123563  259919             1.000  0.143438       0   
99923      164914  202505            -1.500 -0.453926       0   
           192053  298857             0.500 -0.445030       0   
99928      104802  236704             0.000 -1.798462       1   
99936      107913  213906             0.375  1.028336       1   
99937      129380  291004            -0.125 -0.001098       1   
99938      103815  296707             0.375  0.607029       0   
99939      159023  252442             0.375 -1.554640       0   
99944      185654  221067             0.250  0.793300       1   
99946      157197  298613             0.125 -0.643383       0   
99955      108494  243255             1.375 -0.100824       0   
99957      148574  257127            -0.750  0.094124       0   
99965      101083  257338             0.500  0.023504       0   
99966      167228  252173             0.250  0.472597       0   
99973      150202  275083             1.625 -0.008237       1   
99982      112748  275958            -0.625 -0.017792       0   
           151454  221194            -0.500 -0.021966       0   
           183791  239827            -1.500 -0.013180       0   
99983      117390  286606             0.250  0.516310       0   
99985      176670  279638             0.125 -0.477210       0   
99991      151118  226241             1.250 -0.721582       0   
99992      197084  242052             0.000  0.002636       1   
99995      137810  229633            -0.500  0.922350       1   
9

In [97]:
df1.to_csv('data_normalize_median.csv')